# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import library

In [ ]:
from tqdm import tqdm
import os
import cv2
import numpy as np

In [ ]:
def load_data(path):

    images = []
    labels = []

    print("Loading {}".format(path))

    # Iterate through each folder corresponding to a category
    for i, folder in enumerate(os.listdir(path)):
        label = i
        for file in tqdm(os.listdir(os.path.join(path, folder))):

            # Get the path name of the image
            img_path = os.path.join(os.path.join(path, folder), file)

            # Open and resize the img
            image = cv2.imread(img_path, 1)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (224, 224))

            # Append the image and its corresponding label to the output
            images.append(image)
            labels.append(label)

    images = np.array(images, dtype = np.uint8)
    labels = np.array(labels, dtype = np.uint8)

    return (images, labels)

# Load data

In [ ]:
X_train, y_train = load_data("/content/drive/MyDrive/SKRIPSI/dataset/train")
X_val, y_val = load_data("/content/drive/MyDrive/SKRIPSI/dataset/val")
X_test, y_test = load_data("/content/drive/MyDrive/SKRIPSI/dataset/test")

Loading /content/drive/MyDrive/SKRIPSI/dataset/train


100%|██████████| 643/643 [00:19<00:00, 33.61it/s] 


Loading /content/drive/MyDrive/SKRIPSI/dataset/val


100%|██████████| 80/80 [00:49<00:00,  1.60it/s]


Loading /content/drive/MyDrive/SKRIPSI/dataset/test


100%|██████████| 81/81 [00:50<00:00,  1.59it/s]


# Googlenet

## Mendifinisikan model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

class Googlenet(keras.Model):
    def __init__(self):
        pass